# Model Architecture Testing

Test the model architecture with default configuration before optimization.

In [1]:
import sys
import os
from pathlib import Path

# Setup paths
notebook_dir = Path(os.path.abspath('')).parent
build_dir = notebook_dir.parent
sys.path.append(str(notebook_dir))

import pandas as pd
import numpy as np
from src.modeling.architecture import ModelBuilder
from src.preprocessing.preprocessor import FeaturePreprocessor

2025-01-15 18:05:23.214917: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-15 18:05:23.618321: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-15 18:05:23.619479: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-15 18:05:24.665992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load and preprocess data
feature_cols = ['mH2', 'mHD', 'mAD', 'mHDp', 'alpha', 'L2', 'L8', 'vs', 'm22sq']
label_cols = ['valid_BFB', 'valid_Uni', 'valid_STU', 'valid_Higgs']

# Load preprocessor
preprocessor = FeaturePreprocessor.load_transformers(os.path.join(build_dir, 'preprocessor'))

# Load and preprocess train/val sets
train_data = pd.read_csv(os.path.join(build_dir, 'data_splits/train_set.tsv'), sep='\t')
val_data = pd.read_csv(os.path.join(build_dir, 'data_splits/val_set.tsv'), sep='\t')

X_train = preprocessor.transform(train_data[feature_cols])
y_train = train_data[label_cols]

X_val = preprocessor.transform(val_data[feature_cols])
y_val = val_data[label_cols]

In [3]:
# Initialize model builder
builder = ModelBuilder(
    input_shape=(len(feature_cols),),
    num_outputs=len(label_cols)
)

# Get default configuration
config = ModelBuilder.get_default_config()
print("Default configuration:")
for key, value in config.items():
    print(f"{key}: {value}")

Default configuration:
n_layers: 3
n_units_0: 875
n_units_1: 938
n_units_2: 402
activation: relu
dropout_rate: 0.117
apply_batch_norm: True
optimizer: adam
regularization: None
reg_lambda: 0.05
learning_rate: 0.000263


In [4]:
# Build and train model
model = builder.build_model(config)
model.summary()

# Train for a few epochs to verify everything works
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=1024
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 875)               8750      
                                                                 
 activation (Activation)     (None, 875)               0         
                                                                 
 batch_normalization (BatchN  (None, 875)              3500      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 875)               0         
                                                                 
 dense_1 (Dense)             (None, 938)               821688    
                                                                 
 activation_1 (Activation)   (None, 938)               0         
                                                        

2025-01-15 18:06:21.663723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: maienPC
2025-01-15 18:06:21.663741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: maienPC
2025-01-15 18:06:21.663855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2025-01-15 18:06:21.663884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.239.6


Epoch 1/5
80/80 [==============================] - 9s 88ms/step - loss: 0.1342 - subset_accuracy: 0.4462 - hamming_loss: 0.1837 - matthews_correlation: 0.6269 - macro_f1_score: 0.8409 - val_loss: 0.1338 - val_subset_accuracy: 0.3402 - val_hamming_loss: 0.2326 - val_matthews_correlation: 0.5421 - val_macro_f1_score: 0.8260
Epoch 2/5
80/80 [==============================] - 6s 77ms/step - loss: 0.0881 - subset_accuracy: 0.5493 - hamming_loss: 0.1371 - matthews_correlation: 0.7217 - macro_f1_score: 0.8830 - val_loss: 0.1287 - val_subset_accuracy: 0.3498 - val_hamming_loss: 0.2490 - val_matthews_correlation: 0.5429 - val_macro_f1_score: 0.8240
Epoch 3/5
80/80 [==============================] - 6s 79ms/step - loss: 0.0746 - subset_accuracy: 0.6006 - hamming_loss: 0.1179 - matthews_correlation: 0.7609 - macro_f1_score: 0.8993 - val_loss: 0.1366 - val_subset_accuracy: 0.3829 - val_hamming_loss: 0.2240 - val_matthews_correlation: 0.5893 - val_macro_f1_score: 0.8381
Epoch 4/5
80/80 [===========